In [52]:
# Required libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset
import numpy as np
from sklearn.metrics import f1_score
from rouge_score import rouge_scorer

In [53]:
# Load Qwen2.5-0.5B model
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B")

In [ ]:
# 1. First define all helper functions
def categorize_error(entity, entity_type):
    """分析错误类型"""
    if len(entity) <= 1:
        return 'short_entity'
    elif any(title in entity for title in ['公', '王', '君', '尚', '太']):
        return 'title_confusion'
    elif entity_type == 'PER' and any(title in entity for title in ['令', '使', '官']):
        return 'type_confusion'
    elif len(entity.strip()) != len(entity):
        return 'boundary_error'
    return 'other'

In [151]:
def prepare_ner_prompt(text):
    return f"""请识别下面这段古文中的命名实体。

示例输入1：
"太祖启曰：「朱五经平生读书」"

示例输出1：
人名：
太祖
朱五经

地名：

官职：

示例输入2：
"尚书天下尚书，岂独段家尚书也！"

示例输出2：
人名：
段家

地名：

官职：
尚书

注意事项：
1. 每个实体独立成行
2. 实体边界要准确，不要包含多余的称谓词
3. 官职和人名要严格区分
4. 如果某类没有实体，该类别下不需要填写任何内容

现在请识别这段古文中的实体：
{text}

请按相同格式输出实体：

人名：

地名：

官职：
"""

In [152]:
def prepare_summary_prompt(article):
    return f"""请为以下新闻文章生成简短的摘要：

文章：{article}

摘要要求：
1. 保留文章的主要信息
2. 简明扼要
3. 语言通顺自然

请生成摘要："""

def evaluate_summary(prediction, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = scorer.score(prediction, reference)
    return scores

In [153]:
# 测试模型是否加载成功
test_text = "初在缙云山，太极真人徐君降之曰"
prompt = prepare_ner_prompt(test_text)

# 生成回答
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=200)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


请识别下面这段古文中的命名实体。

示例输入1：
"太祖启曰：「朱五经平生读书」"

示例输出1：
人名：
太祖
朱五经

地名：

官职：

示例输入2：
"尚书天下尚书，岂独段家尚书也！"

示例输出2：
人名：
段家

地名：

官职：
尚书

注意事项：
1. 每个实体独立成行
2. 实体边界要准确，不要包含多余的称谓词
3. 官职和人名要严格区分
4. 如果某类没有实体，该类别下不需要填写任何内容

现在请识别这段古文中的实体：
初在缙云山，太极真人徐君降之曰

请按相同格式输出实体：

人名：

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人

地名：

官职：
徐君

地名：

官职：
太极真人



In [154]:
def load_ner_data(file_path):
    sentences = []
    current_sentence = []
    current_tags = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == '':  # 空行表示句子结束
                if current_sentence:
                    sentences.append({
                        'text': ''.join(current_sentence),
                        'tags': current_tags
                    })
                current_sentence = []
                current_tags = []
            else:
                # 分割字符和标签
                parts = line.split('\t')
                if len(parts) == 2:
                    char, tag = parts
                    current_sentence.append(char)
                    current_tags.append(tag)
    
    # 处理最后一个句子
    if current_sentence:
        sentences.append({
            'text': ''.join(current_sentence),
            'tags': current_tags
        })
    
    return sentences

# 测试数据加载
test_data = load_ner_data('data/ner.txt')

# 打印一个示例句子
if test_data:
    print("示例句子：")
    print("文本:", test_data[0]['text'])
    print("标签:", test_data[0]['tags'])
    
    # 统计数据集大小
    print(f"\n总句子数: {len(test_data)}")

示例句子：
文本: 或以问至德，荅曰：「夫庆赏刑罪，人主之权柄，凡为人臣，岂得与人主争权柄哉！」其慎密如此。后高宗知而深歎美之。仪凤四年薨，辍朝三日，使百官以次赴宅哭之，赠开府仪同三司、并州大都督，谥曰恭。
标签: ['O', 'O', 'O', 'B-PER', 'E-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'E-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-OFI', 'I-OFI', 'I-OFI', 'I-OFI', 'I-OFI', 'E-OFI', 'O', 'B-LOC', 'E-LOC', 'B-OFI', 'I-OFI', 'E-OFI', 'O', 'O', 'O', 'S-PER', 'O']

总句子数: 218


In [155]:
def extract_entities(text, tags):
    entities = {
        'PER': [],  # 人名
        'LOC': [],  # 地名
        'OFI': []   # 官职
    }
    
    current_entity = ''
    current_type = ''
    
    for char, tag in zip(text, tags):
        if tag.startswith('B-'):  # 实体开始
            if current_entity:  # 保存之前的实体
                entities[current_type].append(current_entity)
            current_type = tag[2:]  # 获取实体类型（PER/LOC/OFI）
            current_entity = char
        elif tag.startswith('I-') or tag.startswith('E-'):  # 实体中间或结束
            current_entity += char
        elif tag == 'O':  # 非实体
            if current_entity:  # 保存之前的实体
                entities[current_type].append(current_entity)
                current_entity = ''
                current_type = ''
    
    # 处理最后一个实体
    if current_entity:
        entities[current_type].append(current_entity)
    
    return entities

# 测试实体提取
if test_data:
    sample = test_data[0]
    entities = extract_entities(sample['text'], sample['tags'])
    print("\n提取的实体：")
    for entity_type, entity_list in entities.items():
        print(f"{entity_type}: {entity_list}")


提取的实体：
PER: ['至德', '高宗']
LOC: ['并州']
OFI: ['开府仪同三司', '大都督']


In [156]:
def get_model_predictions(text, max_retries=3):
    prompt = prepare_ner_prompt(text)
    
    for attempt in range(max_retries):
        try:
            inputs = tokenizer(prompt, return_tensors="pt")
            outputs = model.generate(
                **inputs,
                max_new_tokens=128,
                do_sample=True,
                temperature=0.3,  # Lower temperature for more focused output
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id,
                early_stopping=True,
                no_repeat_ngram_size=3,
                num_return_sequences=1
            )
            result = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Remove prompt part
            result = result.split("请按相同格式输出实体：")[-1].strip()
            
            # Verify output format
            if "人名：" in result and "地名：" in result and "官职：" in result:
                return result
            else:
                print(f"Attempt {attempt + 1}: Invalid output format")
                continue
                
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == max_retries - 1:
                return ""
            continue
    
    return ""

In [157]:
def parse_model_output(output):
    """将模型输出解析为结构化的实体字典"""
    entities = {
        'PER': [],
        'LOC': [],
        'OFI': []
    }
    
    current_type = None
    lines = output.split('\n')
    for line in lines:
        line = line.strip()
        
        # Skip empty lines and template text
        if not line or '[' in line or ']' in line or '注' in line:
            continue
            
        # Identify entity type
        if '人名：' in line:
            current_type = 'PER'
            continue
        elif '地名：' in line:
            current_type = 'LOC'
            continue
        elif '官职：' in line:
            current_type = 'OFI'
            continue
            
        # Add entity if not a category header
        if current_type and not line.endswith('：'):
            entity = line.strip('- 　').strip()
            if entity and len(entity) > 0:
                entities[current_type].append(entity)
    
    return entities

In [158]:
def calculate_metrics(predicted_entities, true_entities, text=None, tags=None, include_O=False):
    """Calculate precision, recall and F1 score for entity predictions"""
    metrics = {}
    
    # Preprocess entities: standardize format
    def standardize_entities(entities_dict):
        return {
            k: set(e.strip() for e in v if e.strip())
            for k, v in entities_dict.items()
        }
    
    pred_entities = standardize_entities(predicted_entities)
    true_entities = standardize_entities(true_entities)
    
    # Entity types to evaluate
    entity_types = ['PER', 'LOC', 'OFI']
    if include_O and text is not None and tags is not None:
        entity_types.append('O')
        # Add non-entity words to O category
        pred_entities['O'] = set(word for word in text if word not in 
                               [e for ents in pred_entities.values() for e in ents])
        true_entities['O'] = set(word for word, tag in zip(text, tags) if tag == 'O')
    
    for entity_type in entity_types:
        pred_set = pred_entities.get(entity_type, set())
        true_set = true_entities.get(entity_type, set())
        
        tp = len(pred_set & true_set)
        fp = len(pred_set - true_set)
        fn = len(true_set - pred_set)
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        metrics[entity_type] = {
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
    
    return metrics

In [159]:
def evaluate_batch(test_data, num_samples=None):
    """Evaluate multiple samples in batch"""
    if num_samples is None:
        num_samples = len(test_data)
    
    results = {
        'predictions': [],
        'ground_truth': [],
        'metrics_without_O': {
            'PER': {'precision': [], 'recall': [], 'f1': []},
            'LOC': {'precision': [], 'recall': [], 'f1': []},
            'OFI': {'precision': [], 'recall': [], 'f1': []}
        },
        'metrics_with_O': {
            'PER': {'precision': [], 'recall': [], 'f1': []},
            'LOC': {'precision': [], 'recall': [], 'f1': []},
            'OFI': {'precision': [], 'recall': [], 'f1': []},
            'O': {'precision': [], 'recall': [], 'f1': []}
        }
    }
    
    for i, sample in enumerate(test_data[:num_samples]):
        print(f"\n处理样本 {i+1}/{num_samples}")
        
        try:
            # Get predictions and true labels
            prediction = get_model_predictions(sample['text'])
            if not prediction:
                print(f"样本 {i+1} 预测结果为空，跳过")
                continue
                
            parsed_prediction = parse_model_output(prediction)
            true_entities = extract_entities(sample['text'], sample['tags'])
            
            # Store results
            results['predictions'].append(parsed_prediction)
            results['ground_truth'].append(true_entities)
            
            # Calculate metrics without O labels
            metrics_without_O = calculate_metrics(
                parsed_prediction, 
                true_entities,
                include_O=False
            )
            
            # Calculate metrics with O labels
            metrics_with_O = calculate_metrics(
                parsed_prediction,
                true_entities,
                text=sample['text'],
                tags=sample['tags'],
                include_O=True
            )
            
            # Collect both types of metrics
            for entity_type in metrics_without_O:
                for metric in ['precision', 'recall', 'f1']:
                    if entity_type in results['metrics_without_O']:
                        results['metrics_without_O'][entity_type][metric].append(
                            metrics_without_O[entity_type][metric]
                        )
            
            for entity_type in metrics_with_O:
                for metric in ['precision', 'recall', 'f1']:
                    if entity_type in results['metrics_with_O']:
                        results['metrics_with_O'][entity_type][metric].append(
                            metrics_with_O[entity_type][metric]
                        )
            
            print(f"\n样本 {i+1} 结果:")
            print("文本:", sample['text'])
            print("预测:", parsed_prediction)
            print("真实:", true_entities)
            print("不含O标签指标:", metrics_without_O)
            print("含O标签指标:", metrics_with_O)
            
        except Exception as e:
            print(f"处理样本 {i+1} 时发生错误: {e}")
            continue
    
    return results

In [160]:
def test_single_sample(text, true_tags=None):
    """测试单个样本的实体识别效果"""
    print("输入文本:", text)
    print("\n生成的prompt:")
    print(prepare_ner_prompt(text))
    
    print("\n模型输出:")
    prediction = get_model_predictions(text)
    print(prediction)
    
    print("\n解析后的实体:")
    parsed_entities = parse_model_output(prediction)
    print(parsed_entities)
    
    if true_tags is not None:
        print("\n真实实体:")
        true_entities = extract_entities(text, true_tags)
        print(true_entities)
        
        print("\n评估指标:")
        metrics = calculate_metrics(parsed_entities, true_entities)
        for entity_type, scores in metrics.items():
            print(f"{entity_type}:", scores)
    
    return parsed_entities

In [161]:
def run_ner_experiment(test_data, num_samples=None):
    """运行NER实验并记录结果"""
    print("开始NER实验...")
    print(f"数据集大小: {len(test_data)} 样本")
    print(f"实际使用样本数: {num_samples if num_samples else len(test_data)}")
    
    try:
        # 1. 单样本测试
        print("\n1. 单样本测试")
        print("-" * 50)
        sample = test_data[0]
        test_single_sample(sample['text'], sample['tags'])
        
        # 2. 批量评估
        print("\n2. 批量评估")
        print("-" * 50)
        results = evaluate_batch(test_data, num_samples)
        
        # 3. 打印总体结果
        print("\n3. 实验结果总结")
        print("-" * 50)
        
        # 计算并打印平均指标
        for metric_type in ['metrics_without_O', 'metrics_with_O']:
            print(f"\n{metric_type}:")
            for entity_type in results[metric_type]:
                metrics = results[metric_type][entity_type]
                try:
                    avg_metrics = {
                        k: np.mean(v) if v else 0.0 
                        for k, v in metrics.items()
                    }
                    print(f"\n{entity_type}类实体:")
                    print(f"Precision: {avg_metrics['precision']:.4f}")
                    print(f"Recall: {avg_metrics['recall']:.4f}")
                    print(f"F1: {avg_metrics['f1']:.4f}")
                except Exception as e:
                    print(f"计算{entity_type}指标时出错: {e}")
                    continue
        
        # 4. 错误分析
        print("\n4. 错误分析示例")
        print("-" * 50)
        analyze_errors(results, n_examples=3)
        
        return results
        
    except Exception as e:
        print(f"实验运行出错: {e}")
        return None

In [162]:
def calculate_macro_f1(results):
    """Calculate macro F1 score across all entity types"""
    entity_types = ['PER', 'LOC', 'OFI']
    f1_scores = []
    
    # Calculate F1 for each entity type
    for entity_type in entity_types:
        type_metrics = results['metrics_without_O'][entity_type]
        if type_metrics['f1']:  # Only include if we have scores
            avg_f1 = sum(type_metrics['f1']) / len(type_metrics['f1'])
            f1_scores.append(avg_f1)
    
    # Calculate macro F1
    macro_f1 = sum(f1_scores) / len(f1_scores) if f1_scores else 0
    
    return macro_f1

def analyze_errors(results, n_examples=3):
    """Enhanced error analysis with categorization"""
    error_stats = {
        'PER': {'false_positives': 0, 'false_negatives': 0, 'boundary_errors': 0},
        'LOC': {'false_positives': 0, 'false_negatives': 0, 'boundary_errors': 0},
        'OFI': {'false_positives': 0, 'false_negatives': 0, 'boundary_errors': 0}
    }
    
    error_categories = {
        'boundary_issues': 0,
        'title_confusion': 0,
        'missing_short_entities': 0,
        'wrong_entity_type': 0,
        'other': 0
    }
    
    print("详细错误分析:")
    print("-" * 50)
    
    for i, (pred, true) in enumerate(zip(results['predictions'], results['ground_truth'])):
        if i >= n_examples:
            break
            
        print(f"\n示例 {i+1}:")
        for entity_type in ['PER', 'LOC', 'OFI']:
            pred_set = set(pred.get(entity_type, []))
            true_set = set(true.get(entity_type, []))
            
            false_positives = pred_set - true_set
            false_negatives = true_set - pred_set
            
            # Analyze error types
            for entity in false_positives:
                error_type = categorize_error(entity, entity_type)
                error_categories[error_type] += 1
                
            error_stats[entity_type]['false_positives'] += len(false_positives)
            error_stats[entity_type]['false_negatives'] += len(false_negatives)
            
            if false_positives or false_negatives:
                print(f"\n{entity_type} 类实体错误:")
                if false_positives:
                    print(f"错误预测: {false_positives}")
                if false_negatives:
                    print(f"漏检: {false_negatives}")
    
    # Calculate macro F1
    macro_f1 = calculate_macro_f1(results)
    
    print("\n总体性能指标:")
    print("-" * 50)
    print(f"宏平均F1分数: {macro_f1:.4f}")
    
    print("\n错误类型统计:")
    print("-" * 50)
    for category, count in error_categories.items():
        print(f"{category}: {count} 次")
    
    return error_stats, error_categories, macro_f1

def analyze_error_cause(entities, entity_type):
    """分析错误可能的原因"""
    causes = []
    
    for entity in entities:
        if len(entity) <= 1:
            causes.append("实体边界识别不准确")
        if entity_type == 'PER' and ('公' in entity or '王' in entity):
            causes.append("称谓词被错误包含在人名中")
        if entity_type == 'OFI' and len(entity) > 4:
            causes.append("官职边界过长")
            
    return list(set(causes)) if causes else ["需要更多训练样例"]


In [163]:
def print_summary_report(results):
    """Print a comprehensive summary report"""
    error_stats, error_categories, macro_f1 = analyze_errors(results)
    
    print("\n=== 性能评估报告 ===")
    print("\n1. 总体性能:")
    print(f"宏平均F1分数: {macro_f1:.4f}")
    
    print("\n2. 各实体类型性能:")
    for entity_type in ['PER', 'LOC', 'OFI']:
        metrics = results['metrics_without_O'][entity_type]
        avg_precision = sum(metrics['precision']) / len(metrics['precision']) if metrics['precision'] else 0
        avg_recall = sum(metrics['recall']) / len(metrics['recall']) if metrics['recall'] else 0
        avg_f1 = sum(metrics['f1']) / len(metrics['f1']) if metrics['f1'] else 0
        
        print(f"\n{entity_type}:")
        print(f"精确率: {avg_precision:.4f}")
        print(f"召回率: {avg_recall:.4f}")
        print(f"F1分数: {avg_f1:.4f}")
    
    print("\n3. 主要错误类型:")
    for category, count in error_categories.items():
        print(f"{category}: {count} 次")
    
    print("\n4. 改进建议:")
    print("- 加强实体边界识别")
    print("- 改进称谓词处理规则")
    print("- 增加短实体的识别准确率")
    print("- 优化实体类型分类")

In [164]:
# 先用小批量数据测试
print("小批量测试:")
results = run_ner_experiment(test_data, num_samples=5)

# 如果一切正常，运行完整实验
# print("\n完整实验:")
# full_metrics, full_results = run_ner_experiment(test_data)

小批量测试:
开始NER实验...
数据集大小: 218 样本
实际使用样本数: 5

1. 单样本测试
--------------------------------------------------
输入文本: 或以问至德，荅曰：「夫庆赏刑罪，人主之权柄，凡为人臣，岂得与人主争权柄哉！」其慎密如此。后高宗知而深歎美之。仪凤四年薨，辍朝三日，使百官以次赴宅哭之，赠开府仪同三司、并州大都督，谥曰恭。

生成的prompt:
请识别下面这段古文中的命名实体。

示例输入1：
"太祖启曰：「朱五经平生读书」"

示例输出1：
人名：
太祖
朱五经

地名：

官职：

示例输入2：
"尚书天下尚书，岂独段家尚书也！"

示例输出2：
人名：
段家

地名：

官职：
尚书

注意事项：
1. 每个实体独立成行
2. 实体边界要准确，不要包含多余的称谓词
3. 官职和人名要严格区分
4. 如果某类没有实体，该类别下不需要填写任何内容

现在请识别这段古文中的实体：
或以问至德，荅曰：「夫庆赏刑罪，人主之权柄，凡为人臣，岂得与人主争权柄哉！」其慎密如此。后高宗知而深歎美之。仪凤四年薨，辍朝三日，使百官以次赴宅哭之，赠开府仪同三司、并州大都督，谥曰恭。

请按相同格式输出实体：

人名：

地名：

官职：


模型输出:


人名：

地名：

官职：
仪凤

地官职：仪凤
高宗
知
高
仪凤

解析后的实体:
{'PER': [], 'LOC': [], 'OFI': ['仪凤', '高宗', '知', '高', '仪凤']}

真实实体:
{'PER': ['至德', '高宗'], 'LOC': ['并州'], 'OFI': ['开府仪同三司', '大都督']}

评估指标:
PER: {'precision': 0, 'recall': 0.0, 'f1': 0}
LOC: {'precision': 0, 'recall': 0.0, 'f1': 0}
OFI: {'precision': 0.0, 'recall': 0.0, 'f1': 0}

2. 批量评估
--------------------------------------------------

处理样本 1/5

样本 1 结果:
文本: 或以问至德，荅曰：「夫庆赏刑罪，人主之权柄，凡为人臣，岂得与人主争权柄哉！」其慎密如此。后高宗知而深歎美之。仪凤四年薨，辍朝三日，使百官以次赴宅哭之，赠开府仪同三司、并州大都督，谥曰恭。
预测: {'PER': [], 'LOC': [], 'OFI': ['仪凤', '地官员职：仪同', '地员官职', '仪凤', '开府', '仪同', '并州', '大都', '州', '都督', '谥', '或问至徳，荘曰：夫庆賞刑罪人主權柄凡为人子，不得與人主爭權柄也！其慎明如此。後高宗深歊美之。', '或', '问', '至徧', '荅', '曰', '夫', '慶賞', '刑罪', '人主', '權柄', '凡', '為']}
真实: {'PER': ['至德', '高宗'], 'LOC': ['并州'], 'OFI': ['开府仪同三司', '大都督']}
不含O标签指标: {'PER': {'precision': 0, 'recall': 0.0, 'f1': 0}, 'LOC': {'precision': 0, 'recall': 0.0, 'f1': 0}, 'OFI': {'precision': 0.0, 'recall': 0.0, 'f1': 0}}
含O标签指标: {'PER': {'precision': 0, 'recall': 0.0, 'f1':

In [165]:
# def compare_methods(llm_metrics, traditional_metrics):
#     """对比LLM方法和传统方法的性能"""
#     print("性能对比分析:")
#     print("-" * 50)
    
#     for entity_type in ['PER', 'LOC', 'OFI']:
#         print(f"\n{entity_type} 类实体:")
#         print("指标\t\tLLM方法\t传统方法\t差异")
#         print("-" * 50)
        
#         for metric in ['precision', 'recall', 'f1']:
#             llm_score = llm_metrics[entity_type][metric]
#             trad_score = traditional_metrics[entity_type][metric]
#             diff = llm_score - trad_score
            
#             print(f"{metric}\t\t{llm_score:.4f}\t{trad_score:.4f}\t{diff:+.4f}")